<center> 
    <h1> Transformer TTS: A Text-to-Speech Transformer in TensorFlow 2 </h1>
    <h2> Audio synthesis with Forward Transformer TTS and MelGAN Vocoder</h2>
</center>

## Forward Model

In [1]:
# Clone the Transformer TTS and MelGAN repos
!git clone https://github.com/as-ideas/TransformerTTS.git
!git clone https://github.com/seungwonpark/melgan.git

Cloning into 'TransformerTTS'...
remote: Enumerating objects: 140, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 2542 (delta 75), reused 87 (delta 37), pack-reused 2402
Receiving objects: 100% (2542/2542), 7.11 MiB | 28.34 MiB/s, done.
Resolving deltas: 100% (1712/1712), done.
Cloning into 'melgan'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 396 (delta 6), reused 28 (delta 2), pack-reused 356
Receiving objects: 100% (396/396), 18.04 MiB | 34.33 MiB/s, done.
Resolving deltas: 100% (184/184), done.


In [2]:
# Install requirements
!apt-get install -y espeak
!pip install -r TransformerTTS/requirements.txt

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 32 not upgraded.
Need to get 1,219 kB of archives.
After this operation, 3,031 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsonic0 amd64 0.2.0-6 [13.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 espeak-data amd64 1.48.04+dfsg-5 [934 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libespeak1 amd64 1.48.04+dfsg-5 [145 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic/universe amd64 espeak amd64 1.48.04+dfsg-5 [61.6 kB]
Fetched 1,219 kB in 1s (1,363 kB/s)
S

In [3]:
# Download the pre-trained weights
! wget https://public-asai-dl-models.s3.eu-central-1.amazonaws.com/TransformerTTS/ljspeech_melgan_forward_transformer.zip
! unzip ljspeech_melgan_forward_transformer.zip

--2020-06-09 11:19:44--  https://public-asai-dl-models.s3.eu-central-1.amazonaws.com/TransformerTTS/ljspeech_melgan_forward_transformer.zip
Resolving public-asai-dl-models.s3.eu-central-1.amazonaws.com (public-asai-dl-models.s3.eu-central-1.amazonaws.com)... 52.219.47.156
Connecting to public-asai-dl-models.s3.eu-central-1.amazonaws.com (public-asai-dl-models.s3.eu-central-1.amazonaws.com)|52.219.47.156|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 210197051 (200M) [application/zip]
Saving to: ‘ljspeech_melgan_forward_transformer.zip’

ljspeech_melgan_for 100%[===================>] 200.46M  29.7MB/s    in 7.5s    

2020-06-09 11:19:52 (26.8 MB/s) - ‘ljspeech_melgan_forward_transformer.zip’ saved [210197051/210197051]

Archive:  ljspeech_melgan_forward_transformer.zip
   creating: ljspeech_melgan_forward_transformer/
  inflating: ljspeech_melgan_forward_transformer/.DS_Store  
  inflating: __MACOSX/ljspeech_melgan_forward_transformer/._.DS_Store  
   creating

In [0]:
# Set up the paths
from pathlib import Path
MelGAN_path = 'melgan/'
TTS_path = 'TransformerTTS/'
config_path = Path('ljspeech_melgan_forward_transformer/melgan')

import sys
sys.path.append(TTS_path)

In [5]:
# Load pretrained models
from utils.config_manager import ConfigManager
from utils.audio import Audio

import IPython.display as ipd

config_loader = ConfigManager(str(config_path), model_kind='forward')
audio = Audio(config_loader.config)
model = config_loader.load_model(str(config_path / 'forward_weights/ckpt-179'))

restored weights from ljspeech_melgan_forward_transformer/melgan/forward_weights/ckpt-179 at step 895000


In [0]:
# Synthesize text
sentence = 'Scientists at the CERN laboratory, say they have discovered a new particle.'
out_normal = model.predict(sentence)

In [7]:
# Convert spectrogram to wav (with griffin lim)
wav = audio.reconstruct_waveform(out_normal['mel'].numpy().T)
ipd.display(ipd.Audio(wav, rate=config_loader.config['sampling_rate']))

You can also vary the speech speed

In [8]:
# 20% faster
sentence = 'Scientists at the CERN laboratory, say they have discovered a new particle.'
out = model.predict(sentence, speed_regulator=1.20)
wav = audio.reconstruct_waveform(out['mel'].numpy().T)
ipd.display(ipd.Audio(wav, rate=config_loader.config['sampling_rate']))

In [9]:
# 10% slower
sentence = 'Scientists at the CERN laboratory, say they have discovered a new particle.'
out = model.predict(sentence, speed_regulator=.9)
wav = audio.reconstruct_waveform(out['mel'].numpy().T)
ipd.display(ipd.Audio(wav, rate=config_loader.config['sampling_rate']))

### MelGAN

In [10]:
# Do some sys cleaning
sys.path.remove(TTS_path)
sys.modules.pop('model')

<module 'model' from 'TransformerTTS/model/__init__.py'>

In [11]:
sys.path.append(MelGAN_path)
import torch
import numpy as np

vocoder = torch.hub.load('seungwonpark/melgan', 'melgan')
vocoder.eval()

mel = torch.tensor(out_normal['mel'].numpy().T[np.newaxis,:,:])

Downloading: "https://github.com/seungwonpark/melgan/archive/master.zip" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/seungwonpark/melgan/releases/download/v0.3-alpha/nvidia_tacotron2_LJ11_epoch6400.pt" to /root/.cache/torch/hub/checkpoints/nvidia_tacotron2_LJ11_epoch6400.pt


In [0]:
if torch.cuda.is_available():
    vocoder = vocoder.cuda()
    mel = mel.cuda()

with torch.no_grad():
    audio = vocoder.inference(mel)

In [13]:
# Display audio
ipd.display(ipd.Audio(audio.cpu().numpy(), rate=22050))